# Geometryczne - lab 1 - ćwiczenie 1

1. Przygotuj następujące zbiory punktów (2D, współrzędne rzeczywiste typu double):
- a) 10^5 losowych punktów o współrzędnych z przedziału [-1000, 1000],
- b) 10^5 losowych punktów o współrzędnych z przedziału [-10^14 , 10^14 ],
- c) 1000 losowych punktów leżących na okręgu o środku (0,0) i promieniu R=100,
- d) 1000 losowych punktów o współrzędnych z przedziału [-1000, 1000] leżących na prostej
wyznaczonej przez wektor (a, b),
przyjmij a = [-1.0, 0.0], b = [1.0, 0.1].
2. Uruchom wizualizację graficzną utworzonych zbiorów punktów.
3. Przygotuj program, który dla każdego ze zbioru danych dokona podziału punktów względem ich
orientacji w stosunku do odcinka ab ( a = [-1.0, 0.0], b = [1.0, 0.1] – punkty znajdujące się po
lewej stronie, po prawej stronie oraz współliniowe. Obliczenia wykonaj przy pomocy
wyznacznika (1) i następnie (2) zaimplementowanego samodzielnie. Wyszukaj w bibliotekach
numerycznych procedury obliczania wyznacznika 3x3 i 2x2. Dla każdego zbioru danych
porównaj wyniki (podział punktów) uzyskane przy pomocy obu wyznaczników wyliczanych
procedurami własnymi i bibliotecznymi. Określ, ile punktów (i jakich) zostało inaczej
zakwalifikowanych dla różnych sposobów liczenia wyznacznika. Zbadaj wyniki dla różnej
tolerancji dla zera oraz różnych precyzji obliczeń. Odpowiednio zaprezentuj otrzymane wyniki
w tabelach
4. Przedstaw graficznie różnice w podziale punktów

### Narzędzie geometryczne

In [ ]:
####################### GEOMETRIA #######################
# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None
    
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None
    
    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()
####################### GEOMETRIA KONIEC #######################


## Generowanie punktów

In [ ]:
import numpy as np

def gen_points():
    points_a = [ (np.random.uniform(-1000, 1000), np.random.uniform(-1000, 1000)) for _ in range(10**5) ]
    points_b = [ (np.random.uniform(-10**14, 10**14), np.random.uniform(-10**14, 10**14)) for _ in range(10**5) ]
    points_c = []
    for _ in range(1000):
        t = np.random.uniform(-1, 1)
        points_c.append( (np.cos(np.pi*t)*100, np.sin(np.pi*t)*100) )

    points_d = []
    # x = x_0 + mt, y = y_0 + nt
    # x = -1 + 2*t, y = 0 + 0.1*t
    a = (-1, 0)
    b = (1, 0.1)
    v = (2, 0.1)
    for _ in range(1000):
        x = np.random.uniform(-1000, 1000)
        t = (x + 1)/2
        points_d.append( (x, 0.1*t) )
    return points_a, points_b, points_c, points_d

pts_a, pts_b, pts_c, pts_d = gen_points()

### Alternatywnie, punkty już wygenerowane i zserializowane można załadować z pliku

In [ ]:
import pickle

with open('data/points.pickle', 'rb') as f:
    dt = pickle.load(f)

pts_a, pts_b, pts_c, pts_d = dt

## Wizualizacja zbiorów punktów

W razie problemów proszę spróbować zmienić widoczne niżej `%matplotlib widget` na `%matplotlib notebook`

In [ ]:
%matplotlib widget

plot = Plot(points=[PointsCollection(pts_a, s=.5)])
plot.draw()

In [ ]:
plot = Plot(points=[PointsCollection(pts_b, s=.5)])
plot.draw()

In [ ]:
plot = Plot(points=[PointsCollection(pts_c, s=.5)])
plot.draw()

In [ ]:
plot = Plot(points=[PointsCollection(pts_d, s=.5)])
plot.draw()

## Obliczenia

### Wyznaczniki
Uwaga: typ `numpy.float128` jest niedostępny np. na 64-bitowym Windowsie.

In [ ]:
#################### WYZNACZNIKI ####################
import math

# Typ o wysokiej precyzji - dostępny tylko na niektórych platformach
PLATFORM_HAS_HPREC = hasattr(np, 'float128')
if not PLATFORM_HAS_HPREC:
    print('Uwaga: Platforma nie obsługuje typu o wysokiej precyzji.')
    hprec_type = np.float64
else:
    hprec_type = np.float128

def det_2x2(a, b, c, hprec=False):
    if hprec:
        ax, ay = hprec_type(a[0]), hprec_type(a[1])
        bx, by = hprec_type(b[0]), hprec_type(b[1])
        cx, cy = hprec_type(c[0]), hprec_type(c[1])
        return hprec_type((ax-cx)*(by-cy) + (cy-ay)*(bx-cx))
    ax, ay = a
    bx, by = b
    cx, cy = c
    return np.float64((ax-cx)*(by-cy) + (cy-ay)*(bx-cx))

def det_3x3(a, b, c, hprec=False):
    if hprec:
        ax, ay = hprec_type(a[0]), hprec_type(a[1])
        bx, by = hprec_type(b[0]), hprec_type(b[1])
        cx, cy = hprec_type(c[0]), hprec_type(c[1])
        return hprec_type(ax*by + bx*cy + cx*ay - cx*by - ax*cy - bx*ay)
    ax, ay = a
    bx, by = b
    cx, cy = c
    return np.float64(math.fsum([ax*by, bx*cy, cx*ay, -cx*by, -ax*cy, -bx*ay]))

def lib_det_2x2_wrapper(a, b, c, hprec=False):
    if hprec:
        # np.linalg nie wspiera np.float128
        pass
        # ax, ay = hprec_type(a[0]), hprec_type(a[1])
        # bx, by = hprec_type(b[0]), hprec_type(b[1])
        # cx, cy = hprec_type(c[0]), hprec_type(c[1])
        # array = [[ax-cx, ay-cy], [bx-cx, by-cy]]
        # return hprec_type(np.linalg.det(array))
    array = [[a[0]-c[0], a[1]-c[1]], [b[0]-c[0], b[1]-c[1]]]
    return np.float64(np.linalg.det(array))

def lib_det_3x3_wrapper(a, b, c, hprec=False):
    if hprec:
        # np.linalg nie wspiera np.float128
        pass
        # ax, ay = hprec_type(a[0]), hprec_type(a[1])
        # bx, by = hprec_type(b[0]), hprec_type(b[1])
        # cx, cy = hprec_type(c[0]), hprec_type(c[1])
        # array = [[ax, ay, 1], [bx, by, 1], [cx, cy, 1]]
        # return hprec_type(np.linalg.det(array))
    array = [[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]]
    return np.float64(np.linalg.det(array))


### Podział punktów

In [ ]:
SIDES = {1: 'left', 0: 'coll', -1: 'right'}

def side_to_str(rc):
    return SIDES[rc]


def which_side(a, b, c, eps, det_fun, hprec=False):
    res = det_fun(a, b, c, hprec)
    if res > eps:
        return 1
    if res < -eps:
        return -1
    return 0


A = (-1, 0)
B = (1, 0.1)

def group_points(points, eps=10**(-8), det_fun=det_2x2, hprec=False):
    res = [[],[],[]]
    for c in points:
        res[which_side(A, B, c, eps, det_fun, hprec)+1].append(c)
    return res


def do_math(custom_dets=None, hprec=False):
    if hprec and not PLATFORM_HAS_HPREC:
        print('Uwaga: Platforma nie obsługuje typu o wysokiej precyzji.')
        return {}
    points_d = {'A': pts_a, 'B': pts_b, 'C': pts_c, 'D': pts_d}
    dets_d = {'own det 2x2': det_2x2, 'own det 3x3': det_3x3, 'lib det 2x2': lib_det_2x2_wrapper, 'lib det 3x3': lib_det_3x3_wrapper}
    eps_d = {'10^-10': 10**(-10), '10^-12': 10**(-12), '10^-14': 10**(-14)}
    if hprec:
        dets_d.pop('lib det 2x2')
        dets_d.pop('lib det 3x3')
        eps_d['10^-16'] = hprec_type(10**(-16))
        eps_d['10^-18'] = hprec_type(10**(-18))
    results = {}
    for pts_str, pts in points_d.items():
        result_pts = {}
        if custom_dets is not None:
            dets_d = custom_dets
        for det_str, det in dets_d.items():
            result_pts_det = {}
            if hprec:
                det_str += ' (HP)'
            for eps_str, eps in eps_d.items():
                result_pts_det[eps_str] = group_points(pts, eps=eps, det_fun=det, hprec=hprec)
            result_pts[det_str] = result_pts_det
        results[pts_str] = result_pts
    return results

## Mierzenie czasu wykonania funkcji

In [ ]:
import timeit

d = {'own det 2x2': det_2x2}
t = timeit.Timer("do_math(d)", setup="from __main__ import do_math, d")
t.repeat(number=10)

In [ ]:
d = {'own det 3x3': det_3x3}
t = timeit.Timer("do_math(d)", setup="from __main__ import do_math, d")
t.repeat(number=10)

In [ ]:
d = {'lib det 2x2': lib_det_2x2_wrapper}
t = timeit.Timer("do_math(d)", setup="from __main__ import do_math, d")
t.repeat(number=10)

In [ ]:
d = {'lib det 3x3': lib_det_3x3_wrapper}
t = timeit.Timer("do_math(d)", setup="from __main__ import do_math, d")
t.repeat(number=10)

## Prezentacja danych

### Definicja funkcji prezentującej dane

In [ ]:
def present_data(*args):
    for results in args:
        for pts_str, results_pts in results.items():
            print(f'[Points set {pts_str}]')
            for det_str, results_pts_det in results_pts.items():
                print(f'[Using {det_str}]')
                print('eps\tleft\tcoll\tright')
                for eps_str, entry in results_pts_det.items():
                    print(eps_str, end='\t')
                    for l in entry[::-1]:
                        print(len(l), end='\t')
                    print()
                print()
            print()

### Definicje funkcji serializujących dane i wczytujących je z pliku

In [ ]:
import pickle
import os
def dump_to_file(*args, path='data/'):
    for i, results in enumerate(args):
        if not os.path.exists(path):
            os.mkdir(path)
        with open(os.path.join(path, f'results{i+1}.pickle'), 'wb') as out_file:
            pickle.dump(results, out_file)

def load_from_file(fpath):
    if not os.path.isfile(fpath):
        return {}
    with open(fpath, 'rb') as file:
        res = pickle.load(file)
    return res

### Obliczanie wyników

In [ ]:
RESULTS = do_math()
RESULTS_HP = do_math(hprec=True)

### Wczytywanie wyników z pliku

In [ ]:
RESULTS = load_from_file('data/results1.pickle')
RESULTS_HP = load_from_file('data/results2.pickle')

### Prezentacja danych

Proszę wcisnąć `Open the full output data in a text editor` aby zobaczyć pełne wyjście funkcji

In [ ]:
present_data(RESULTS, RESULTS_HP)

### Definicje funkcji prezentujących wizualizacje wyników

In [ ]:
COLOUR_LEFT='#88AAFF'
COLOUR_COLL='#11FF11'
COLOUR_RIGHT='#FF8888'
COLOUR_LINE='#000000'

def get_line_points(a, b, x_rng):
    # v = (b[0]-a[0], b[1]-a[1])
    ax, ay = a
    bx, by = b
    y = lambda x: (ay-by)/(ax-bx)*x + (ay-(ay-by)/(ax-bx)*ax)
    return [(x_rng[0], y(x_rng[0])), (x_rng[1], y(x_rng[1]))]

def display_tricolour_points(result, order='lrc', x_rng=None, distinct=''):
    pts = []
    for clr in order:
        if clr == 'l':
            pts.append(PointsCollection(result[2], color=COLOUR_LEFT, s=.5))
        if clr == 'c':
            sz = 10 if 'c' in distinct else .5
            pts.append(PointsCollection(result[1], color=COLOUR_COLL, s=sz, marker='o'))
        if clr == 'r':
            pts.append(PointsCollection(result[0], color=COLOUR_RIGHT, s=.5))
    lines = [LinesCollection([get_line_points(A, B, x_rng)], color=COLOUR_LINE, linewidth=.5)] if x_rng is not None else []
    plot = Plot(lines=lines, points=pts)
    plot.draw()

def display_compound_data(results):
    eps_order = ('10^-10', '10^-12', '10^-14')
    colours_order = (('midnightblue', 'maroon'), ('blue', 'red'), ('royalblue', 'tomato'))
    left_used = set()
    right_used = set()
    pts = []
    for eps, (colour_left, colour_right) in zip(eps_order, colours_order):
        result = results[eps]
        left_new = [x for x in result[2] if x not in left_used]
        right_new = [x for x in result[0] if x not in right_used]
        pts.append(PointsCollection(left_new, color=colour_left, s=.5))
        pts.append(PointsCollection(right_new, color=colour_right, s=.5))
        left_used.update(left_new)
        right_used.update(right_new)
    plot = Plot(points=pts)
    plot.draw()

### Wizualizacje wyników

In [ ]:
display_tricolour_points(RESULTS['A']['own det 2x2']['10^-10'], x_rng=(-1000, 1000))

In [ ]:
display_tricolour_points(RESULTS['B']['own det 3x3']['10^-10'], x_rng=(-10**14, 10**14))

In [ ]:
display_tricolour_points(RESULTS['B']['own det 2x2']['10^-10'], x_rng=(-10**14, 10**14), distinct='c')

In [ ]:
display_tricolour_points(RESULTS['B']['lib det 2x2']['10^-10'], x_rng=(-10**14, 10**14), distinct='c')

In [ ]:
display_tricolour_points(RESULTS['C']['own det 2x2']['10^-10'], x_rng=(-100, 100))

In [ ]:
display_tricolour_points(RESULTS['D']['own det 2x2']['10^-10'], 'crl')

In [ ]:
display_compound_data(RESULTS['D']['own det 2x2'])

In [ ]:
display_compound_data(RESULTS['D']['lib det 3x3'])

In [ ]:
display_compound_data(RESULTS['D']['lib det 2x2'])

### Poniższe wizualizacje dostępne wyłącznie jeśli policzono wyniki z wyższą precyzją

In [ ]:
display_tricolour_points(RESULTS_HP['D']['own det 2x2 (HP)']['10^-16'], 'clr')

In [ ]:
display_tricolour_points(RESULTS_HP['D']['own det 3x3 (HP)']['10^-16'], 'clr')